# MNIST Classification using Keras

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/truefoundry-examples/blob/main/mnist-classification-keras/train.ipynb)

### Install packages and import

In [ ]:
%pip install -U "mlfoundry>=0.3.33,<0.4.0"

In [ ]:
import mlfoundry as mlf

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import Callback


### Initialize MLFoundry run

In [ ]:
from getpass import getpass
api_token = getpass("TrueFoundry API Token (Get it from https://app.truefoundry.com/settings):")

In [ ]:
mlf_api = mlf.get_client(api_key=api_token)

In [ ]:
run = mlf_api.create_run('mnist-keras')

### Split train and test data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Define the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

### Define the callback to log metrics to MLFoundry

In [ ]:
class MetricsLogCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        run.log_metrics(logs, epoch)  

### Train the model

In [ ]:
BATCH_SIZE = 128
EPOCHS = 5

run.log_params({'batch_size': BATCH_SIZE, 'epochs': EPOCHS})

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1, callbacks=[MetricsLogCallback()])

### Log the model and end run

In [ ]:
run.log_model(model, 'keras')
run.end()
print(run.run_id)